In [1]:
import os
from utils.dataprocess.vibration_data_loader import VibrationDataLoader  
from utils.dataprocess.start_idx_visualized_select import StartIdxVisualizedSelect
from utils.dataprocess.debiasing import debias_csv_folder
from utils.visualize.data_processor import DataProcessor
from utils.visualize.visualization_generator import VisualizationGenerator
from loguru import logger
import sys
from utils.visualize.extract_timepoint import extract_timepoint_data, visualize_grid,export_filename_grid
import pandas as pd
import matplotlib
# Set up the matplotlib backend explicitly
matplotlib.use('TkAgg')  # Use TkAgg backend which has good button support

In [30]:
# 配置日志记录
logger.configure(handlers=[{"sink": sys.stdout, "level": "INFO"},
                           {"sink": "logs/log.log", "level": "INFO"}])

# 确保输出目录存在
os.makedirs('output', exist_ok=True)
os.makedirs('logs', exist_ok=True)
os.makedirs('output/data_csv', exist_ok=True)
os.makedirs('output/data_csv_start-idx-reselected', exist_ok=True)
os.makedirs('output/data_csv_start-idx-reselected_debiased', exist_ok=True)

In [ ]:
# 1. 使用VibrationDataLoader将TXT文件转换为CSV
logger.info("开始将TXT文件转换为CSV格式")
VibrationDataLoader.convert_txt_to_csv_batch(
    "./input/data",  # 包含TXT文件的输入文件夹
    "./output/data_csv"  # CSV文件的输出文件夹
)

In [ ]:
# 2. 使用StartIdxVisualizedSelect进行起始点选择
logger.info("开始进行数据起始点选择")
processor_idx = StartIdxVisualizedSelect(
    "./output/data_csv",  # 包含CSV文件的输入文件夹
    "./output/data_csv_start-idx-reselected"  # 处理后文件的输出文件夹
)
processor_idx.run()

In [ ]:
# 3. 使用debias_csv_folder进行数据去偏处理
logger.info("开始进行数据去偏处理")
debias_csv_folder(
    "./output/data_csv_start-idx-reselected",  # 选择起始点后的输入文件夹 
    "./output/data_csv_start-idx-reselected_debiased",  # 去偏后数据的输出文件夹
    truncate_to_min=True  # 将所有文件截断至最短文件的长度
)



In [ ]:
# 4. 使用DataProcessor处理数据并组织为网格结构
logger.info("开始处理数据并组织为网格结构")
processor = DataProcessor(
    input_folder="./output/data_csv_start-idx-reselected_debiased",  # 修改为去偏后的数据目录
    rows=6,  # 根据数据布局调整行数
    cols=6,  # 根据数据布局调整列数
    sampling_points=500,  # 采样点数量
    use_all_points=False  # 是否使用所有点,默认是False,如果为True,则使用所有点，忽略采样点数量
)
processor_use_all_points = DataProcessor(
    input_folder="./output/data_csv_start-idx-reselected_debiased",  # 修改为去偏后的数据目录
    rows=6,  # 根据数据布局调整行数
    cols=6,  # 根据数据布局调整列数
    use_all_points=True  # 是否使用所有点,默认是False,如果为True,则使用所有点，忽略采样点数量
)
# 5. 保存处理后的数据
processor.save_processed_data("./output/my_processed_data.npz")
logger.info("数据已处理并保存到./output/my_processed_data.npz")
processor_use_all_points.save_processed_data("./output/my_processed_data_use_all_points.npz")
logger.info("数据已处理并保存到./output/my_processed_data_use_all_points.npz")



In [ ]:
# 6. 创建可视化
logger.info("开始生成可视化")
processed_data = processor.get_processed_data()


# 分析数据中的最小值和最大值
grid_data = processed_data['grid_data']
data_min = grid_data.min()
data_max = grid_data.max()
logger.info(f"数据范围: {data_min:.6f} 到 {data_max:.6f}")

# 定义所有可视化使用的统一颜色范围，可以先不设置，后面每个具体的图单独设置
viz_gen = VisualizationGenerator(
    processed_data=processed_data,
    colormap="viridis",  # 颜色映射
    output_folder="./output",  # 输出文件夹
    vmin=None,  # 设置颜色范围最小值，None表示使用现有数据的最小值
    vmax=None   # 设置颜色范围最大值，None表示使用现有数据的最大值
)


In [ ]:
# 7. 生成指定时间点的热图
specific_time = 0.5  # 指定0.5秒时刻
viz_gen.generate_heatmap_at_time(
    target_time=specific_time,
    output_file=None, # 指定热图的输出文件名，可以不指定，默认是"heatmap_time_{target_time:.4f}_{timestamp}.png"
    title=None, # 指定热图的标题，可以不指定，默认是“Signal Intensity at Specific \n Time :t=0.50s”
    vmin=None,  # 指定颜色范围最小值，None表示使用现有数据的最小值
    vmax=None   # 指定颜色范围最大值，None表示使用现有数据的最大值
)

In [ ]:
# 8. 生成指定时间点和角度的3D表面图
logger.info("生成指定时间点的3D表面图")
viz_gen.generate_3d_surface_at_time(
    target_time=specific_time,
    output_file=None, # 指定3D表面图的输出文件名，可以不指定，默认是"3d_surface_time_{target_time:.4f}_{timestamp}.png"
    title=f"时刻 {specific_time:.2f}s 3D表面图",  # 指定3D表面图的标题
    vmin=None,  # 指定颜色范围最小值，None表示使用现有数据的最小值
    vmax=None   # 指定颜色范围最大值，None表示使用现有数据的最大值
)

In [ ]:
# 9. 生成不旋转的指定角度的3D表面视频
logger.info("生成固定视角的3D表面视频")
viz_gen.generate_3d_surface_video(
    rotate_view=False,  # 不旋转视角
    initial_elev=45,   # 初始俯仰角度
    initial_azim=45,   # 初始方位角度
    add_timestamp=True,  # 添加时间戳
    vmin=None,  # 指定颜色范围最小值，None表示使用现有数据的最小值
    vmax=None,   # 指定颜色范围最大值，None表示使用现有数据的最大值
    output_file="surface3d_fixed_view_elev45_azim45.mp4",
    title="固定视角的3D表面动画"
)


In [ ]:
# 10. 生成热图视频
logger.info("生成热图视频")
viz_gen.generate_heatmap_video(
    output_file="heatmap_animation.mp4",
    title="振动信号热图动画",
    add_timestamp=True,  # 添加时间戳
    add_colorbar=True,   # 添加颜色条
    vmin=None,  # 指定颜色范围最小值，None表示使用现有数据的最小值
    vmax=None   # 指定颜色范围最大值，None表示使用现有数据的最大值
)

In [ ]:
# 11. 生成带剖面图的热图视频
logger.info("生成带剖面图的热图视频")
viz_gen.generate_heatmap_with_profiles_video(
    output_file="heatmap_with_profiles.mp4",
    title="带剖面图的热图动画",
    add_timestamp=True,  # 添加时间戳
    vmin=None,  # 指定颜色范围最小值，None表示使用现有数据的最小值
    vmax=None   # 指定颜色范围最大值，None表示使用现有数据的最大值
)

In [ ]:
# 12. 使用extract_timepoint提取特定时间点的数据
logger.info("使用extract_timepoint提取特定时间点的数据")

# 12.1 提取特定时间点的数据
# 这里注意为了更好的时间分辨率，这里使用my_processed_data_use_all_points.npz，
# 而不是my_processed_data.npz，因为my_processed_data.npz的采样点数量是500，
# 而my_processed_data_use_all_points.npz使用了所有的原始数据点
extracted_time, extracted_data, metadata = extract_timepoint_data(
    "./output/my_processed_data_use_all_points.npz",
    target_time=1.0  # 目标时间点为1.0秒
)


# 12.2 可视化提取的数据
fig = visualize_grid(
    extracted_time,
    extracted_data,
    title=f"时刻 {extracted_time:.2e}s 的数据",
    filename_grid=metadata.get('filename_grid'),
    vmin=None,  # 指定颜色范围最小值，None表示使用现有数据的最小值
    vmax=None   # 指定颜色范围最大值，None表示使用现有数据的最大值
)

# 保存图像
fig.savefig(f"output/extracted_time_{extracted_time:.2e}s.png", dpi=300)
logger.info(f"已保存时间点 {extracted_time:.2e}s 的提取数据可视化")

# 12.3 将提取的数据保存为CSV
# 构建保存的数据
csv_data = pd.DataFrame(extracted_data)
csv_file = f"./output/extracted_time_{extracted_time:.2e}s.csv"
csv_data.to_csv(csv_file)

logger.info(f"已保存时间点 {extracted_time:.2e}s 的数据到CSV: {csv_file}")

# 保存filename_grid
export_filename_grid(metadata, f"./output/extracted_time_{extracted_time:.2e}s_filename_grid.csv")
logger.info("数据处理和可视化完成") 